In [261]:
#Importing the libraries
import cv2
import numpy as np

In [262]:
img = cv2.imread("D:\iitkgp.jpg")

img1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("Original_Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [263]:
#Generate the Gaussian 5x5 kernel
def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g

In [264]:
gf = gaussian_kernel(5, 2)
img2 = cv2.filter2D(img1, -1, gf)
cv2.imshow("Image_After_Noise_Reduction", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [265]:
from scipy import ndimage

def sobel_filters(img):
    Kx = np.array([[-0.1, 0, 0.1], [-0.2, 0, 0.2], [-0.1, 0, 0.1]], np.float32)
    Ky = np.array([[0.1, 0.2, 0.1], [0, 0, 0], [-0.1, -0.2, -0.1]], np.float32)
    
    Ix = ndimage.filters.convolve(img, Kx)
    Iy = ndimage.filters.convolve(img, Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)

In [266]:
img3, thetha = sobel_filters(img2)
img3 = img3.astype(np.uint8)
cv2.imshow("Edges_After_Gradient_Calculation", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21452\4219078671.py:7: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Ix = ndimage.filters.convolve(img, Kx)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21452\4219078671.py:8: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  Iy = ndimage.filters.convolve(img, Ky)


In [267]:
#Non-Maximum Suppression
def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z

In [268]:
img4 = non_max_suppression(img3, thetha)
img4 = img4.astype(np.uint8)
cv2.imshow("Edges_After_NonMaxSuppression", img4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [269]:
#Double Threshold
def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
    
    highThreshold = img.max() * highThresholdRatio;
    lowThreshold = highThreshold * lowThresholdRatio;
    
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    
    weak = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(img >= highThreshold)
    zeros_i, zeros_j = np.where(img < lowThreshold)
    
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return (res, weak, strong)

In [270]:
img5, weak, strong = threshold(img4)
img5 = img5.astype(np.uint8)
cv2.imshow("Edges_After_DoubleThreshold", img5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [271]:
#Edge Tracking by Hysteresis
def hysteresis(img, weak, strong=255):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [272]:
img6 = hysteresis(img5, weak, strong)
img6 = img6.astype(np.uint8)
cv2.imshow("Final_Image", img6)
cv2.waitKey(0)
cv2.destroyAllWindows()